## DIA 053: Creacion de un Dashboard con Flask y Chart.js para Visualizacion de Datos

Hoy desarrollaremos un dashboard interactivo en Flask utilizando Chart.js para visualizar datos en tiempo real.
Este dashboard permitirá a los administradores visualizar métricas como:
✅ Cantidad de usuarios registrados 📊
✅ Predicciones realizadas en la API 🚀
✅ Tendencias de uso en el tiempo 📈

El objetivo es crear una API que sirva datos dinámicamente y una interfaz web con gráficos interactivos.

🖥️ Código Completo (app.py)
python
Copiar
Editar
import os
import json
import logging
from datetime import datetime, timedelta
from flask import Flask, render_template, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate

# Configuración básica
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'supersecretkey')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos de la Base de Datos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

class Prediction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction_result = db.Column(db.String(120))
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

# ---------------------------
# Rutas del Dashboard
# ---------------------------
@app.route('/')
def dashboard():
    """Renderiza la página del dashboard."""
    return render_template("dashboard.html")

@app.route('/api/data')
def get_data():
    """Proporciona datos en formato JSON para el dashboard."""
    
    # Contar usuarios registrados
    user_count = User.query.count()

    # Contar predicciones realizadas
    prediction_count = Prediction.query.count()

    # Obtener predicciones en los últimos 7 días
    last_week = datetime.utcnow() - timedelta(days=7)
    daily_predictions = db.session.query(db.func.date(Prediction.timestamp), db.func.count()) \
        .filter(Prediction.timestamp >= last_week) \
        .group_by(db.func.date(Prediction.timestamp)).all()
    
    # Formatear datos para JSON
    labels = [str(date) for date, _ in daily_predictions]
    values = [count for _, count in daily_predictions]

    return jsonify({
        "users": user_count,
        "predictions": prediction_count,
        "labels": labels,
        "values": values
    })

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
🎨 Código HTML del Dashboard (templates/dashboard.html)
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dashboard - Visualización de Datos</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>
    <h1>Dashboard de Uso</h1>
    
    <h3>Usuarios Registrados: <span id="userCount">0</span></h3>
    <h3>Predicciones Realizadas: <span id="predictionCount">0</span></h3>

    <canvas id="predictionChart"></canvas>

    <script>
        async function fetchData() {
            const response = await fetch('/api/data');
            const data = await response.json();
            
            document.getElementById("userCount").innerText = data.users;
            document.getElementById("predictionCount").innerText = data.predictions;

            const ctx = document.getElementById("predictionChart").getContext("2d");
            new Chart(ctx, {
                type: "line",
                data: {
                    labels: data.labels,
                    datasets: [{
                        label: "Predicciones diarias",
                        data: data.values,
                        borderColor: "blue",
                        borderWidth: 2,
                        fill: false
                    }]
                },
                options: {
                    responsive: true,
                    scales: {
                        y: {
                            beginAtZero: true
                        }
                    }
                }
            });
        }

        fetchData();
    </script>
</body>
</html>
🔍 Explicación de las Principales Implementaciones
🔹 📊 API para Datos del Dashboard (/api/data)

Retorna el número total de usuarios y predicciones realizadas.
Calcula la cantidad de predicciones por día en la última semana.
Formatea los datos en JSON para que sean usados por el frontend.
🔹 📜 Página Web Interactiva (dashboard.html)

Usa Chart.js para graficar tendencias de uso.
Muestra en tiempo real la cantidad de usuarios y predicciones.
Realiza una solicitud AJAX para actualizar los datos.
🔹 📂 Base de Datos con SQLAlchemy

Se crean modelos para usuarios y predicciones.
Se usa una consulta con group_by para calcular predicciones por día.
🔹 🔄 Datos en Tiempo Real

El frontend obtiene datos de /api/data y los muestra dinámicamente.
No es necesario recargar la página para ver nuevas estadísticas.